In [1]:
import numpy as np
import pandas as pd
import re
import spacy
import jsonlines
from spacy.symbols import nsubj, VERB
from spacy.pipeline import EntityRuler
from spacy.matcher import Matcher
from spacy.tokens import Span


In [40]:
nlp = spacy.load("pt_core_news_sm")

In [3]:
df = pd.read_excel('LinkedIn jobs complete.xlsx')
texts = df['description']
job = df['job']

In [4]:
def preprocessing(texts_old):
    new_texts = []

    for text in texts_old:
        new_text = text.replace('.','. ')
        new_text = new_text.replace(',',', ')
        new_text = new_text.replace(';','. ')
        new_text = new_text.replace(':','. ')
        new_text = new_text.replace('/','. ')
        new_text = new_text.replace('\n','. ')
        new_text = new_text.replace(' . ',' ')
        new_text = new_text.replace('  ',' ')
        new_texts.append(new_text)
    return new_texts

In [5]:
texts = preprocessing(texts)

In [6]:
doc = nlp(texts[0])

In [7]:
print(doc)
for t in doc:
    
    print(t,t.tag_)

.      A Creditas é a plataforma líder na América Latina de crédito e soluções 100% online, construída a partir de três ecossistemas. imobiliário, veículo e salário. A empresa trabalha ativamente para mudar o cenário do mercado de crédito no Brasil e no México por meio da experiência com o usuário e da tecnologia proprietária, que foram desenvolvidos para ajudar seus clientes a atingirem seus objetivos. Sua carteira de empréstimos inclui os Empréstimos com Garantia de Imóvel, de Veículo e Consignado Privado, que também viabilizam o acesso a diferentes soluções e serviços com opções integradas de financiamento e refinanciamento. . Fundada em 2012 e hoje localizada nos escritórios em São Paulo, Barueri, Curitiba, Porto Alegre, Recife, Valência (Espanha) e Cidade do México. Recebeu aportes de fundos de capital de risco internacionais, que somam mais de US$570 milhões e foi reconhecida pelos rankings KPMG, Business Insider e Glassdoor como uma das fintechs mais promissoras do mundo. . Noss

Além ADV
disso PRON
, PUNCT
também ADV
oferecemos NOUN
auxílio-creche ADJ
. PUNCT
  SPACE
Diversidade PROPN
e CCONJ
Inclusão PROPN
na ADP
Creditas PROPN
. PUNCT
Você PRON
é AUX
livre ADJ
para SCONJ
ser AUX
você PRON
! PUNCT
Aqui ADV
valorizamos VERB
a DET
pluralidade NOUN
e CCONJ
equidade NOUN
de ADP
raça NOUN
, PUNCT
cor NOUN
, PUNCT
religião NOUN
, PUNCT
identidade NOUN
de ADP
gênero NOUN
, PUNCT
nacionalidade NOUN
, PUNCT
deficiência NOUN
, PUNCT
orientação NOUN
sexual ADJ
, PUNCT
ascendência NOUN
ou CCONJ
idade NOUN
. PUNCT
Para ADP
nós PRON
, PUNCT
ser AUX
plural ADJ
é AUX
valorizar VERB
e CCONJ
respeitar VERB
as DET
diferenças NOUN
com ADP
empatia NOUN
e CCONJ
ter VERB
equidade NOUN
significa VERB
promover VERB
e CCONJ
garantir VERB
oportunidades NOUN
e CCONJ
condições NOUN
justas ADJ
para ADP
todas DET
as DET
pessoas NOUN
e CCONJ
grupos NOUN
. PUNCT
Com ADP
backgrounds NOUN
e CCONJ
perspectivas NOUN
diferentes ADJ
, PUNCT
a DET
nossa DET
Tripulação PROPN
consegue VERB
entender V

In [8]:
matcher = Matcher(nlp.vocab)

pattern1 = [
     [{"LOWER": {"IN": ["fazer", "realizar",'apoiar','facilidade','validar','consolidar','acompanhar']}},
      {'POS':{'IN':['VERB','ADP','DET','NOUN','ADJ','PROPN']},'OP': '+'},
      {'IS_PUNCT':True}]]

pattern2 = 

    [[
        {"LOWER": {"IN":["formação", "conhecimento",'experiencia','habilidades','habilidade','experiência','completo','entendimento','facilidade']}},
        {'POS':'ADP'},
        {'POS':{'IN':['VERB','ADP','DET','NOUN','ADJ','PROPN']},'OP': '+'},
     
     {'IS_PUNCT':True}
    ]]

pattern3 = [
    [{'POS':{'IN':
             ['VERB','NOUN','ADJ','PROPN']}},
    {'LOWER':{'IN':
             ['avançado','intermediário','intermediario','basico','básico']}},
    {'IS_PUNCT':False,'OP':'*'},
    {'IS_PUNCT':True}]]


matcher.add('job description',pattern1)
matcher.add('skills',pattern2)
matcher.add('short skills',pattern3)

In [9]:
matches = matcher(doc)
n = 0
for match_id, start, end in matches:
    n=+1
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(job[n],string_id, span.text,'\n')

Analista de Controladoria Pleno|Brasilseg|São Paulo, Brasil|Presencial|69 ex-alunos trabalham aqui|Promovida|Candidate-se facilmente job description acompanhar o planejamento orçamentário do negócio, 

Analista de Controladoria Pleno|Brasilseg|São Paulo, Brasil|Presencial|69 ex-alunos trabalham aqui|Promovida|Candidate-se facilmente job description Realizar consolidação contábil para demonstrações financeiras. 

Analista de Controladoria Pleno|Brasilseg|São Paulo, Brasil|Presencial|69 ex-alunos trabalham aqui|Promovida|Candidate-se facilmente skills Entendimento de normas contábeis em IFRS. 

Analista de Controladoria Pleno|Brasilseg|São Paulo, Brasil|Presencial|69 ex-alunos trabalham aqui|Promovida|Candidate-se facilmente short skills Inglês avançado. 



In [10]:
data = {'job':[],'tipo':[],'description':[]}
n = 0

for text in texts:
    doc = nlp(text)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        data['job'].append(job[n])
        data['tipo'].append(string_id)
        data['description'].append(span.text)
    
    n+=1
    
            

In [11]:
df = pd.DataFrame(data)

In [12]:
df['cargo'] = df['job'].str.split('|',expand=True)[0]
df['empresa'] = df['job'].str.split('|',expand=True)[1]
df['local'] = df['job'].str.split('|',expand=True)[2]

In [13]:
df.drop(['job'],axis=1,inplace=True)

In [14]:
df['description'] = df['description'].str.replace('[^\w\s]','')

In [15]:
#df.to_excel('LinkedIn skills.xlsx')

In [16]:
df.describe()

,tipo,description,cargo,empresa,local
count,1624,1624,1549,1549,1549
unique,3,1110,250,372,122
top,skills,Excel avançado,Analista de Controladoria,Expresso Vagas,"São Paulo, São Paulo, Brasil"
freq,862,53,142,74,341


In [17]:
df_onlyskills = df.drop(df[df['tipo'] == 'job description'].index)

In [18]:
df_onlyskills.describe()

,tipo,description,cargo,empresa,local
count,1216,1216,1155,1155,1155
unique,2,812,236,350,116
top,skills,Excel avançado,Analista de Controladoria,Expresso Vagas,"São Paulo, São Paulo, Brasil"
freq,862,53,102,57,265


In [44]:
pattern23 = [{"label": "skills", "pattern":[
    
    [{"LOWER": {"IN": 
                ["formação", "conhecimento",'experiencia','habilidades','habilidade','experiência','completo','entendimento','facilidade']
               }},
     
     {'POS':'ADP'},
     {'POS':{'IN':['VERB','ADP','DET','NOUN','ADJ','PROPN']},'OP': '+'},
     
     {'IS_PUNCT':True}]]}]

In [47]:
#ruler = nlp.add_pipe("entity_ruler", config={"validate": True})
#ruler.add_patterns(pattern23)
ruler.patterns

[{'label': 'habilidades',
  'pattern': [[{'LOWER': {'IN': ['formação',
       'conhecimento',
       'experiencia',
       'habilidades',
       'habilidade',
       'experiência',
       'completo',
       'entendimento',
       'facilidade']}},
    {'POS': 'ADP'},
    {'POS': {'IN': ['VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'PROPN']}, 'OP': '+'},
    {'IS_PUNCT': True}]]},
 {'label': 'habilidades',
  'pattern': [[{'LOWER': {'IN': ['formação',
       'conhecimento',
       'experiencia',
       'habilidades',
       'habilidade',
       'experiência',
       'completo',
       'entendimento',
       'facilidade']}},
    {'POS': 'ADP'},
    {'POS': {'IN': ['VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'PROPN']}, 'OP': '+'},
    {'IS_PUNCT': True}]]},
 {'label': 'skills',
  'pattern': [[{'LOWER': {'IN': ['formação',
       'conhecimento',
       'experiencia',
       'habilidades',
       'habilidade',
       'experiência',
       'completo',
       'entendimento',
       'facilidade']}},
    {'POS'

In [49]:
doc = nlp(texts[50])
print([(ent.text, ent.label_) for ent in doc.ents])

[('Beneficios', 'PER'), ('Formação Acadêmica', 'PER'), ('Salário', 'PER'), ('Cargo', 'LOC'), ('Empresa', 'MISC'), ('Humano', 'LOC'), ('Empresarial', 'LOC'), ('Ramo', 'PER'), ('Recursos Humanos', 'PER')]


In [50]:
ruler.to_disk("./patterns.jsonl")

Preprocessing for the model